In [21]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# Set the device for CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Load test data
test_data = pd.read_csv('../../../../mnt/sda/suhohan/emnist/emnist-byclass-test.csv')
num_test_samples = test_data.shape[0]

# Prepare the test data
x_test = test_data.iloc[:, 1:].to_numpy().reshape((num_test_samples, 28, 28, 1))
x_test_resized = tf.image.resize(x_test, [32, 32])
x_test_rgb = tf.repeat(x_test_resized, 3, axis=3)
y_test = tf.keras.utils.to_categorical(test_data.iloc[:, 0], 62)
# Indexing to get a specific example

# Model loading
model = tf.keras.models.load_model('./checkpoints_resnet/weights.1714736836.7261658.hdf5')
loss_resnet, acc_resnet = model.evaluate(x_test_rgb, y_test)

# Results output
print("Loss:", loss_resnet)
print("Accuracy:", acc_resnet)


3636/3636 [==============================] - 40s 11ms/step - loss: 0.3384 - accuracy: 0.8727
Loss: 0.33835020661354065
Accuracy: 0.8726637959480286


In [22]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# Set the device for CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Load test data
test_data = pd.read_csv('../../../../mnt/sda/suhohan/emnist/emnist-byclass-test.csv')
num_test_samples = test_data.shape[0]

# Prepare the test data
x_test = test_data.iloc[:, 1:].to_numpy().reshape((num_test_samples, 28, 28, 1))
y_test = tf.keras.utils.to_categorical(test_data.iloc[:, 0], 62)
# Indexing to get a specific example

# Model loading
model = tf.keras.models.load_model('./checkpoints_lenet/weights.1714736836.7261548.hdf5')
loss_lenet, acc_lenet = model.evaluate(x_test, y_test)

# Results output
print("Loss:", loss_lenet)
print("Accuracy:", acc_lenet)


3636/3636 [==============================] - 11s 3ms/step - loss: 0.3666 - accuracy: 0.8653
Loss: 0.36661601066589355
Accuracy: 0.8652705550193787
